In [1]:
import langchain
print(langchain.__version__)

0.3.26


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq
import os

# Get the API key from the environment
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    api_key=groq_api_key
)

In [4]:
llm_response = llm.invoke("Tell me about cricket")

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Cricket is a popular team sport played with a bat and ball on a rectangular field with two sets of three stumps (wickets) at each end. The game is widely played in many countries, particularly in the Commonwealth nations, and is known for its rich history, complex rules, and passionate fan base.\n\n**Objective:**\nThe objective of cricket is simple: score runs by hitting the ball with a bat, while the opposing team tries to stop you by getting you out. The team with the most runs at the end of the game wins.\n\n**Basic Rules:**\n\n1. **The Game:** A cricket match is played between two teams, each with 11 players. The game is divided into innings, with each team taking turns to bat and bowl.\n2. **The Pitch:** The game takes place on a rectangular field with a flat surface, known as the pitch. The pitch has a length of 22 yards (20.12 meters) and a width of 3.33 yards (3.04 meters).\n3. **The Wickets:** At each end of the pitch, there are three stumps (called wickets) stuck into the gr

In [6]:
chain = llm | output_parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nI hope that made you laugh! Do you want to hear another one?"

In [7]:
chain = llm | output_parser
chain.invoke("Tell me a cat")

"Once upon a time, in a cozy little house on a quiet street, there lived a beautiful cat named Whiskers. Whiskers was a stunning feline with shimmering grey fur, bright green eyes, and a fluffy tail that she loved to swish back and forth whenever she was excited or curious.\n\nWhiskers spent her days lounging in the sunbeams that streamed through the windows, chasing the occasional fly, and purring contentedly as her owner, a kind old lady named Mrs. Jenkins, petted her and gave her treats.\n\nOne day, while Mrs. Jenkins was out in the garden, Whiskers decided to explore the house. She padded silently down the hallway, her ears perked up and her tail twitching, as she sniffed and prowled through each room.\n\nAs she wandered, Whiskers stumbled upon a hidden door that she had never seen before. The door was small and tucked away behind a bookshelf, and it looked like it hadn't been opened in years. Whiskers' curiosity was piqued, and she decided to investigate.\n\nShe batted at the door

In [8]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['gorgeous screen', 'great camera', 'solid battery life'], cons=['expensive', 'no charger included', 'new button layout can be confusing'], summary='A solid phone with a few annoying quirks, great for those due for an upgrade')

In [9]:
output.pros

['gorgeous screen', 'great camera', 'solid battery life']

In [10]:
output.rating

4.0

In [11]:
output.cons

['expensive', 'no charger included', 'new button layout can be confusing']

In [12]:
output.summary

'A solid phone with a few annoying quirks, great for those due for an upgrade'

In [13]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
prompt.invoke({"topic": "programming"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about programming', additional_kwargs={}, response_metadata={})])

In [14]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "programer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs.'

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    api_key=groq_api_key
)

# Define the output parser
output_parser = StrOutputParser()

# Compose the chain
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "football"})
print(result)

Why did the football go to the doctor?

Because it was feeling a little "deflated"!


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about programming")
llm.invoke([system_message, human_message])

AIMessage(content='Programming, the art of telling computers what to do, one line of code at a time. It\'s like writing a recipe for a robot, but instead of ingredients, you use variables, loops, and functions.\n\nBut don\'t worry, programming isn\'t as scary as it sounds. In fact, it\'s quite "byte-sized" (get it?). With the right tools and a bit of practice, you can create anything from a simple calculator to a complex game.\n\nThere are many programming languages to choose from, each with its own strengths and weaknesses. Some popular ones include:\n\n* Python: Known for its simplicity and ease of use, Python is a great language for beginners.\n* Java: A popular choice for Android app development and web applications.\n* JavaScript: Used for client-side scripting on the web, JavaScript is a must-know for any aspiring web developer.\n* C++: A powerful language for building operating systems, games, and other high-performance applications.\n\nBut programming isn\'t just about writing 

In [17]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "quantum mechanics"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about quantum mechanics', additional_kwargs={}, response_metadata={})])

In [18]:
llm.invoke(prompt_value)

AIMessage(content='Quantum mechanics, the branch of physics that\'s so weird, it\'s like a joke! But don\'t worry, I\'ll try to explain it in a way that\'s not too "wave"-y.\n\nQuantum mechanics is a fundamental theory that describes the behavior of matter and energy at the smallest scales, like atoms and subatomic particles. It\'s a realm where the usual rules of classical physics don\'t apply, and things get really strange.\n\nImagine you\'re at a party, and you\'re trying to find your friend. In classical physics, you\'d look around the room, and when you find your friend, you\'d know exactly where they are and how fast they\'re moving. But in quantum mechanics, it\'s like your friend is playing a game of hide-and-seek, and they can be in many places at the same time!\n\nThis is called superposition, where a quantum particle can exist in multiple states simultaneously. It\'s like being in two places at once, which is pretty cool, but also a bit confusing.\n\nAnother weird aspect of 

In [19]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docx_loader = PyPDFLoader("./HR_Documents/Tesla Leave and attendance policies.pdf")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

38
Split the documents into 67 chunks.


In [20]:
documents[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-07-06T00:20:19-07:00', 'author': 'Chandan H A', 'moddate': '2025-07-06T00:20:19-07:00', 'source': './HR_Documents/Tesla Leave and attendance policies.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}, page_content='Tesla, Inc. \nLeave and Attendance Policy Manual \nEffective Date: January 1, 2025 \n Version: 3.2 \n Document Classification: Internal Use Only \n Owner: Human Resources Department \n Review Cycle: Annual \n \nTable of Contents \n1. Introduction and Purpose \n2. Scope and Applicability \n3. General Attendance Requirements \n4. Work Schedules and Flexible Work Arrangements \n5. Time Tracking and Recording \n6. Vacation and Paid Time Off (PTO) \n7. Sick Leave \n8. Family and Medical Leave (FMLA) \n9. Personal Leave of Absence \n10. Bereavement Leave \n11. Jury Duty and Court Appearances \n12. Military Leave \n13. Parental Leave \n14. Educational Leave \n15. Sabbatical Leave \n16. R

In [21]:
splits[1]

Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-07-06T00:20:19-07:00', 'author': 'Chandan H A', 'moddate': '2025-07-06T00:20:19-07:00', 'source': './HR_Documents/Tesla Leave and attendance policies.pdf', 'total_pages': 38, 'page': 1, 'page_label': '2'}, page_content="20. Manager Responsibilities \n21. Employee Rights and Resources \n22. Policy Violations and Consequences \n23. Appendices \n \n1. Introduction and Purpose \n1.1 Company Mission Statement \nTesla's mission is to accelerate the world's transition to sustainable energy. Our success \ndepends on the dedication, creativity, and reliability of our team members. This Leave and \nAttendance Policy is designed to support work-life balance while maintaining operational \nexcellence and ensuring fair treatment for all employees. \n1.2 Policy Objectives \nThis policy establishes guidelines for: \n• Maintaining consistent attendance standards across all Tesla locations \n• Providing comprehen

In [22]:
splits[0].metadata

{'producer': 'PyPDF',
 'creator': 'Microsoft Word',
 'creationdate': '2025-07-06T00:20:19-07:00',
 'author': 'Chandan H A',
 'moddate': '2025-07-06T00:20:19-07:00',
 'source': './HR_Documents/Tesla Leave and attendance policies.pdf',
 'total_pages': 38,
 'page': 0,
 'page_label': '1'}

In [23]:
splits[0].page_content

'Tesla, Inc. \nLeave and Attendance Policy Manual \nEffective Date: January 1, 2025 \n Version: 3.2 \n Document Classification: Internal Use Only \n Owner: Human Resources Department \n Review Cycle: Annual \n \nTable of Contents \n1. Introduction and Purpose \n2. Scope and Applicability \n3. General Attendance Requirements \n4. Work Schedules and Flexible Work Arrangements \n5. Time Tracking and Recording \n6. Vacation and Paid Time Off (PTO) \n7. Sick Leave \n8. Family and Medical Leave (FMLA) \n9. Personal Leave of Absence \n10. Bereavement Leave \n11. Jury Duty and Court Appearances \n12. Military Leave \n13. Parental Leave \n14. Educational Leave \n15. Sabbatical Leave \n16. Religious Observances \n17. Domestic Violence Leave \n18. Attendance Management and Progressive Discipline \n19. Return to Work Procedures'

In [24]:
# 1. Function to load documents from a folder

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "./HR_Documents"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 87 documents from the folder.
Split the documents into 156 chunks.


In [25]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# 4. Embedding Documents
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
print(f"Created embeddings for {len(document_embeddings)} document chunks.")

d:\CCC\Langchain_RAG\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Created embeddings for 156 document chunks.


In [26]:
document_embeddings[0]

[-0.021179858595132828,
 0.036873847246170044,
 -0.04491506889462471,
 -0.008675085380673409,
 0.03825949504971504,
 0.03315889090299606,
 0.07473839819431305,
 0.03474042937159538,
 0.025133799761533737,
 -0.039531003683805466,
 0.10581167042255402,
 0.042663250118494034,
 -0.015336505137383938,
 0.007082195021212101,
 0.004827227909117937,
 -0.018377047032117844,
 -0.006810976192355156,
 -0.021477367728948593,
 0.019223975017666817,
 0.008177065290510654,
 -0.03638341277837753,
 0.0157979317009449,
 -0.009980439208447933,
 0.005969990976154804,
 0.025276198983192444,
 -0.029804876074194908,
 0.06785968691110611,
 0.006770770531147718,
 0.08408190310001373,
 0.020279165357351303,
 0.05136492848396301,
 -0.0037406133487820625,
 0.02938203327357769,
 0.02721506357192993,
 2.12776762964495e-06,
 0.009790587238967419,
 -0.030335770919919014,
 0.011450511403381824,
 0.007383040618151426,
 -0.01691523939371109,
 -0.03574887663125992,
 0.01832769066095352,
 -0.00614004023373127,
 0.045297920

In [27]:
# !pip install sentence_transformers
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
document_embeddings[0]

C:\Users\Chandan H A\AppData\Local\Temp\ipykernel_9400\3719339062.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


[0.004566734656691551,
 0.06250039488077164,
 0.03443426266312599,
 0.06747285276651382,
 0.020967168733477592,
 0.13422024250030518,
 -0.05751041695475578,
 -0.06238238513469696,
 -0.053967852145433426,
 -0.015135458670556545,
 0.0833054929971695,
 0.06373736262321472,
 0.016117535531520844,
 -0.03385379537940025,
 0.022591831162571907,
 -0.03747020661830902,
 0.07084092497825623,
 -0.06782300770282745,
 0.013225491158664227,
 -0.021245846524834633,
 -0.024841921404004097,
 0.02910015732049942,
 -0.009938068687915802,
 0.03727283328771591,
 0.05986284837126732,
 0.029413286596536636,
 -0.026982711628079414,
 0.018668195232748985,
 -0.026007235050201416,
 0.012578463181853294,
 0.03650091588497162,
 0.024131648242473602,
 -0.04982318356633186,
 -0.009216631762683392,
 0.020163901150226593,
 0.03897895663976669,
 0.034384727478027344,
 0.019170530140399933,
 -0.011178823187947273,
 -0.031325627118349075,
 0.010492680594325066,
 -0.05754132941365242,
 0.018529215827584267,
 -0.0561426021

In [28]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdf_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [29]:
# Step 6: Add documents to vector store
vector_store.add_documents(splits)

['6591117a-0224-4838-86c3-29cfcdd3f118',
 '7dc075fd-3f08-43be-84da-787121abc41a',
 '6fe51046-5360-4477-800a-e6a2c0890a39',
 '1f072d55-b040-4606-a836-93e948bccff5',
 'f58bae53-0587-4899-ae65-3f5c8e5005bf',
 '239d2780-7702-4a9f-be60-5a6a249c5f74',
 '405f2568-5285-4149-85c3-55e052f6eef4',
 'e986e40d-99fc-4e52-8db5-5182a9b411ee',
 'd71363be-4488-43b9-b34f-725e812cdcf4',
 'a858ebb1-0266-48a1-8257-a754a5b59d1e',
 'b4a8cf9d-a11f-48ce-9bda-7512ac2f5143',
 '5fed43d0-4e66-45fd-90fc-e2aac89da98a',
 '2c11c208-e26c-4943-8d34-8802a2d8f2d8',
 '3cf53b4f-ae3f-479e-8d02-56cf6bbe719e',
 '2447d36b-69cf-4198-a04b-08c63a2b3ad1',
 '045331c8-25ab-4d5e-b890-15f41a898f5c',
 '47f37523-7db2-4458-bd98-49f81f28ac25',
 '278c4039-9a59-4f62-a2be-9709796082d8',
 '05d18f34-fdfa-4308-9b9d-a87373da8475',
 '555c4f2b-5f9b-4661-8927-c31f342de191',
 'c32adaf4-7e87-4fdf-9a46-85f4a776867d',
 '09dc4108-3410-45ec-8663-339c662bb7a0',
 '31768cd5-0825-45fc-95ad-b8fc812d62e2',
 '26dce0a6-5f2d-4760-8387-d414e36d9c81',
 '1fae4a55-0b97-

In [30]:
# 5. Perform similarity search

query = "Explain in detail about the maternity leave policy?"
search_results = vector_store.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'Explain in detail about the maternity leave policy?'

Result 1:
Source: ./HR_Documents\Tesla Leave and attendance policies.pdf
Content: • Leave requests and approval/denial decisions 
• Attendance tracking and patterns 
• Performance discussions related to attendance 
• Accommodation requests and implementations 
• Medical documentation handling (confidentially) 
20.4 Avoiding Discrimination and Retaliation 
Best practices: 
• Treat all employees equally regardless of protected status 
• Do not discourage legitimate leave usage 
• Focus on job performance, not leave usage 
• Consult HR before any disciplinary action 
Warning signs to avoid: 
• Comments about pregnancy, disability, or family status 
• Questioning legitimacy of protected leave 
• Retaliatory actions after leave return 
• Inconsistent policy application 
20.5 Manager Training and Support 
• Annual training on leave policies and legal compliance 
• Regular updates on policy chang

In [31]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})
retriever.invoke("Explain in detail about Sabbatical Leave")

[Document(id='ae59a479-7181-4445-b327-38aa2baa1b30', metadata={'producer': 'PyPDF', 'page_label': '21', 'author': 'Chandan H A', 'creationdate': '2025-07-06T00:20:19-07:00', 'creator': 'Microsoft Word', 'page': 20, 'source': './HR_Documents\\Tesla Leave and attendance policies.pdf', 'moddate': '2025-07-06T00:20:19-07:00', 'total_pages': 38}, page_content='• Minimum 7 years of continuous employment \n• Consistent high performance ratings \n• Approved sabbatical proposal with clear objectives \n• Department head and executive approval \n15.3 Sabbatical Types \nResearch and Development:'),
 Document(id='2bc37283-e65d-4d50-be97-6c1d7a9a40e6', metadata={'page': 29, 'creator': 'Microsoft Word', 'moddate': '2025-07-06T00:20:19-07:00', 'creationdate': '2025-07-06T00:20:19-07:00', 'producer': 'PyPDF', 'author': 'Chandan H A', 'source': './HR_Documents\\Tesla Leave and attendance policies.pdf', 'total_pages': 38, 'page_label': '30'}, page_content='• Leave requests and approval/denial decisions \

In [32]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("Explain in detail about sabatical leave policy?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(id='2bc37283-e65d-4d50-be97-6c1d7a9a40e6', metadata={'page': 29, 'creationdate': '2025-07-06T00:20:19-07:00', 'source': './HR_Documents\\\\Tesla Leave and attendance policies.pdf', 'page_label': '30', 'author': 'Chandan H A', 'total_pages': 38, 'moddate': '2025-07-06T00:20:19-07:00', 'producer': 'PyPDF', 'creator': 'Microsoft Word'}, page_content='• Leave requests and approval/denial decisions \\n• Attendance tracking and patterns \\n• Performance discussions related to attendance \\n• Accommodation requests and implementations \\n• Medical documentation handling (confidentially) \\n20.4 Avoiding Discrimination and Retaliation \\nBest practices: \\n• Treat all employees equally regardless of protected status \\n• Do not discourage legitimate leave usage \\n• Focus on job performance, not leave usage \\n• Consult HR before any disciplinary action \\nWarning signs to avoid:

In [34]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [35]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("Explain in detail about sabatical leave policy?")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\n• Leave requests and approval/denial decisions \n• Attendance tracking and patterns \n• Performance discussions related to attendance \n• Accommodation requests and implementations \n• Medical documentation handling (confidentially) \n20.4 Avoiding Discrimination and Retaliation \nBest practices: \n• Treat all employees equally regardless of protected status \n• Do not discourage legitimate leave usage \n• Focus on job performance, not leave usage \n• Consult HR before any disciplinary action \nWarning signs to avoid: \n• Comments about pregnancy, disability, or family status \n• Questioning legitimacy of protected leave \n• Retaliatory actions after leave return \n• Inconsistent policy application \n20.5 Manager Training and Support \n• Annual training on leave policies and legal compliance \n• Regular updates on policy changes \n• HR consultation availability \n• Resources for di

In [36]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "Explain about  Religious Observances"
response = rag_chain.invoke(question)
print(response)

According to the context, Tesla has a Religious Accommodation Policy that respects and accommodates employees' religious observances and practices. The policy includes:

1. **Time off** for religious holidays and observances, such as:
	* Religious holidays not included in the company calendar
	* Weekly observances (e.g., Sabbath, Friday prayers)
	* Religious festivals and special occasions
2. **Flexible scheduling** for religious practices, including:
	* Earlier start/end times for observances
3. **Workplace modifications** for religious requirements
4. **Dietary accommodations** in company facilities

To request a religious accommodation, employees must:

1. Provide **advance notice**, preferably with an annual calendar of regular observances or a minimum of 2 weeks' notice for known dates.
2. Submit a **written request** describing the accommodation needed.
3. Provide **religious organization verification** if requested.
4. Discuss the request with their **supervisor and HR**.

The g

In [37]:
# Example conversation
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [38]:
chat_history

[HumanMessage(content='Explain about  Religious Observances', additional_kwargs={}, response_metadata={}),
 AIMessage(content="According to the context, Tesla has a Religious Accommodation Policy that respects and accommodates employees' religious observances and practices. The policy includes:\n\n1. **Time off** for religious holidays and observances, such as:\n\t* Religious holidays not included in the company calendar\n\t* Weekly observances (e.g., Sabbath, Friday prayers)\n\t* Religious festivals and special occasions\n2. **Flexible scheduling** for religious practices, including:\n\t* Earlier start/end times for observances\n3. **Workplace modifications** for religious requirements\n4. **Dietary accommodations** in company facilities\n\nTo request a religious accommodation, employees must:\n\n1. Provide **advance notice**, preferably with an annual calendar of regular observances or a minimum of 2 weeks' notice for known dates.\n2. Submit a **written request** describing the accom

In [39]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "Explain in detail about  Domestic Violence Leave", "chat_history": chat_history})

"Domestic Violence Leave is a type of leave policy that allows employees to take time off from work to address and recover from domestic violence situations. The policy aims to support and protect employees who are experiencing or have experienced domestic violence, providing them with a safe and secure environment to seek help and rebuild their lives.\n\n**Purpose of Domestic Violence Leave:**\n\nThe primary purpose of Domestic Violence Leave is to provide employees with a supportive and non-judgmental environment to:\n\n1. Seek medical attention and counseling\n2. Obtain a restraining order or other legal protection\n3. Find safe and alternative housing\n4. Attend court proceedings or other legal appointments\n5. Address other related needs, such as childcare or financial assistance\n\n**Eligibility and Entitlement:**\n\nEmployees who are experiencing or have experienced domestic violence are eligible for Domestic Violence Leave. The entitlement typically includes:\n\n1. A specified 

In [40]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "Paid Leaves up to how many days?", "chat_history": chat_history})

[Document(id='c32adaf4-7e87-4fdf-9a46-85f4a776867d', metadata={'source': './HR_Documents\\Tesla Leave and attendance policies.pdf', 'creator': 'Microsoft Word', 'page': 11, 'moddate': '2025-07-06T00:20:19-07:00', 'page_label': '12', 'total_pages': 38, 'author': 'Chandan H A', 'producer': 'PyPDF', 'creationdate': '2025-07-06T00:20:19-07:00'}, page_content='8.4 Leave Duration and Usage \nStandard FMLA: \n• Up to 12 weeks in a 12-month rolling period \n• May be taken continuously or intermittently \n• Intermittent leave subject to business needs approval \nMilitary Caregiver Leave: \n• Up to 26 weeks in a single 12-month period \n• Combined with other FMLA usage cannot exceed 26 weeks \n8.5 FMLA Application Process \nEmployee Responsibilities: \n1. Provide 30-day advance notice when foreseeable \n2. Complete FMLA application forms \n3. Provide medical certification within 15 calendar days \n4. Submit periodic recertification as required \n5. Provide fitness-for-duty certification before r

In [41]:
retriever.invoke("Explain in detail Progressive Discipline Steps")

[Document(id='cef65328-6d78-457b-8be0-c7b253745692', metadata={'page_label': '6', 'creationdate': '2025-07-06T00:19:50-07:00', 'total_pages': 49, 'source': './HR_Documents\\Tesla Performance review guidelines.pdf', 'creator': 'Microsoft Word', 'producer': 'PyPDF', 'author': 'Chandan H A', 'page': 5, 'moddate': '2025-07-06T00:19:50-07:00'}, page_content="• Quarterly: Formal progress reviews and goal adjustments \n• Mid-year: Comprehensive progress assessment and recalibration \n3.2 Review Timeline Details \n3.2.1 Pre-Review Phase (8 weeks) \nWeeks 1-2: Goal Setting and Self-Assessment \n• Employees complete comprehensive self-assessments \n• Managers review previous year's goals and set new objectives \n• HR provides review timeline and resource access \nWeeks 3-4: 360-Degree Feedback Collection \n• Peer nominations and feedback requests submitted \n• Direct reports provide upward feedback \n• Cross-functional partners contribute input \nWeeks 5-6: Manager Assessment Period \n• Managers

In [42]:
retriever.invoke("Explain in detail about Return to Work Procedures")

[Document(id='b57d1b48-346c-4b4a-bf13-d700b1cf78c2', metadata={'source': './HR_Documents\\Tesla Leave and attendance policies.pdf', 'total_pages': 38, 'creator': 'Microsoft Word', 'creationdate': '2025-07-06T00:20:19-07:00', 'author': 'Chandan H A', 'page_label': '28', 'page': 27, 'moddate': '2025-07-06T00:20:19-07:00', 'producer': 'PyPDF'}, page_content='• Return date and any limitations specified \nAccommodation assessment: \n• Review of any ongoing restrictions or limitations \n• Reasonable accommodation evaluation \n• Job modification possibilities \n• Alternative position consideration if needed \n19.3 FMLA Return Procedures \nTimely return: \n• Return by date specified in leave approval \n• Contact HR if return date changes \n• Provide updated medical certification if extended \nPosition restoration: \n• Same position if available \n• Equivalent position with same pay and benefits \n• Cannot be better or worse than original position \n19.4 Extended Leave Returns \nPosition availa

In [43]:
import random

fallback_responses = [
    "I'm here to assist with Tesla's HR-related topics like leave policies, employee benefits, or onboarding procedures. For anything outside that, you may want to check with another department. How can I help with your HR questions?",
    "I’m really focused on helping with HR policies at Tesla. I might not have accurate info outside that scope, but I’d be happy to help with anything HR-related!",
    "That’s outside my current scope — I specialize in Tesla’s HR topics like time-off policies, reimbursements, and onboarding. Would you like to ask something about those?",
    "That’s an interesting question! But I’ve been trained only on Tesla’s HR-related topics like benefits, leaves, and company policies. Is there something I can help with in those areas?",
    "I'm here to help with HR policies at Tesla. Let me know how I can assist you on that front!",
    "I couldn't find any verified information about that topic in the HR documents. I recommend checking with the appropriate department. Meanwhile, feel free to ask about HR-related topics!",
    "That’s outside what I can assist with based on HR documentation. You might consider reaching out to the relevant team, or let me know if there’s something I can help with regarding HR at Tesla!"
]

# Choose one randomly for this invocation
fallback_response = random.choice(fallback_responses)

In [44]:
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')  # Format: '2025-05-14'

In [45]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     f"""You are an AI assistant for Tesla, a software solutions and services company. 
Your role is to help employees with questions about HR policies and internal documentation.

If someone asks a casual or friendly question like "How are you?" or "What are you doing?", respond politely and conversationally.

Today is {today}. Use this date when answering any date-related questions such as holidays or leave calculations.

If someone asks a question that is **not related** to Tesla's HR policies or employee matters, respond with:
"{fallback_response}"

When responding to HR-related questions:
- Only answer based on verified information from company documents.
- Do not make up answers or include guesses.
- Avoid using technical terms like 'context' in your response.
- If something is not mentioned in the documents, simply state that the information is not available or refer the user to HR.

Guidelines:
- Provide clear, detailed, and helpful answers.
- Use all available information from the documents to support and elaborate your response.
- Maintain a professional and supportive tone.
- If you do not find the answer in the provided context, say "I couldn't find that information in the HR documents."
- If the answer is not in the context, do not answer. Instead, say: "I couldn't find that information in the HR documents. Please reach out to the HR department for assistance."
- Avoid making up answers or including guesses."""
    ),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [46]:
rag_chain.invoke({"input": "Explain in detail about Bereavement Leave", "chat_history":chat_history})

{'input': 'Explain in detail about Bereavement Leave',
 'chat_history': [HumanMessage(content='Explain about  Religious Observances', additional_kwargs={}, response_metadata={}),
  AIMessage(content="According to the context, Tesla has a Religious Accommodation Policy that respects and accommodates employees' religious observances and practices. The policy includes:\n\n1. **Time off** for religious holidays and observances, such as:\n\t* Religious holidays not included in the company calendar\n\t* Weekly observances (e.g., Sabbath, Friday prayers)\n\t* Religious festivals and special occasions\n2. **Flexible scheduling** for religious practices, including:\n\t* Earlier start/end times for observances\n3. **Workplace modifications** for religious requirements\n4. **Dietary accommodations** in company facilities\n\nTo request a religious accommodation, employees must:\n\n1. Provide **advance notice**, preferably with an annual calendar of regular observances or a minimum of 2 weeks' noti

In [47]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [48]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "Explain in detail about Bereavement Leave"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-4-o-mini")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: Explain in detail about Bereavement Leave
AI: Bereavement Leave is a type of leave provided by Tesla to support employees who have experienced the loss of a loved one. The company understands that this can be a difficult and emotional time, and the leave is intended to allow employees to take the time they need to grieve and attend to personal matters.

**Leave Provision:**
Tesla provides up to 3 days of paid leave for funeral attendance. This leave can be taken to attend the funeral or memorial service of a close personal friend or family member, as determined by the supervisor's discretion. In addition to the 3 days of paid leave, one additional day may be granted for travel if the funeral or memorial service is held at a distant location.

**Notification:**
If an employee needs to take Bereavement Leave, they should inform their supervisor as soon as possible. The employee should also provide an expected duration of absence, which will help the supervisor to plan and make 

In [49]:
question2 = "Eligible family members"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'Explain in detail about Bereavement Leave'}, {'role': 'ai', 'content': "Bereavement Leave is a type of leave provided by Tesla to support employees who have experienced the loss of a loved one. The company understands that this can be a difficult and emotional time, and the leave is intended to allow employees to take the time they need to grieve and attend to personal matters.\n\n**Leave Provision:**\nTesla provides up to 3 days of paid leave for funeral attendance. This leave can be taken to attend the funeral or memorial service of a close personal friend or family member, as determined by the supervisor's discretion. In addition to the 3 days of paid leave, one additional day may be granted for travel if the funeral or memorial service is held at a distant location.\n\n**Notification:**\nIf an employee needs to take Bereavement Leave, they should inform their supervisor as soon as possible. The employee should also provide an expected duration of abse

In [50]:
question3 = "Explain about Family and Medical Leave"
chat_history = get_chat_history(session_id)
print(chat_history)
answer3 = rag_chain.invoke({"input": question3, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question3, answer3, "gpt-3.5-turbo")
print(f"Human: {question3}")
print(f"AI: {answer3}\n")

[{'role': 'human', 'content': 'Explain in detail about Bereavement Leave'}, {'role': 'ai', 'content': "Bereavement Leave is a type of leave provided by Tesla to support employees who have experienced the loss of a loved one. The company understands that this can be a difficult and emotional time, and the leave is intended to allow employees to take the time they need to grieve and attend to personal matters.\n\n**Leave Provision:**\nTesla provides up to 3 days of paid leave for funeral attendance. This leave can be taken to attend the funeral or memorial service of a close personal friend or family member, as determined by the supervisor's discretion. In addition to the 3 days of paid leave, one additional day may be granted for travel if the funeral or memorial service is held at a distant location.\n\n**Notification:**\nIf an employee needs to take Bereavement Leave, they should inform their supervisor as soon as possible. The employee should also provide an expected duration of abse

In [51]:
session_id = str(uuid.uuid4())
question = "Explain about Sick Leave procedure"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: Explain about Sick Leave procedure
AI: The Sick Leave procedure at Tesla involves several steps to ensure a fair and consistent application of leave policies while maintaining operational effectiveness. Here's an overview of the process:

1. **Written Request**: Employees are required to submit a written request detailing the reason for their sick leave, proposed duration, and return date. They must also provide supporting documentation as applicable.
2. **Plan for Work Coverage**: Employees should include a plan for work coverage during their absence to ensure a smooth transition of tasks and minimal disruption to the team.
3. **Submission to Direct Supervisor**: The employee must submit their request to their direct supervisor for review.
4. **HR Review and Recommendation**: The supervisor will forward the request to HR for review and recommendation.
5. **Department Head Approval**: The department head will review and approve or deny the request.
6. **Final Approval by Regi